In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import logging
import os
import pickle as pkl
import pprint
import time
import numpy as np

import torch
import torch.nn.parallel
import torch.optim
from torch.utils.collect_env import get_pretty_env_info
from tensorboardX import SummaryWriter

import _init_paths
from config import config
from config import update_config
from core.function import inference, test
from core.loss import build_criterion
from dataset import build_dataloader
from dataset import RealLabelsImagenet
from models import build_model
from utils.comm import comm
from utils.utils import create_logger
from utils.utils import init_distributed
from utils.utils import setup_cudnn
from utils.utils import summary_model_on_master
from utils.utils import strip_prefix_if_present

In [11]:
parser = argparse.ArgumentParser()
parser.add_argument('--cfg',
                        help='experiment configure file name',
                        default=r'D:\desktop\1\z\feature\transformer\original-global-plus-local-fp-transformer\experiments\global_plus_local-inference.yaml' ,
                        type=str)

    # distributed training
parser.add_argument("--local_rank", type=int, default=0)
parser.add_argument("--port", type=int, default=9000)

parser.add_argument('opts',
                    help="Modify config options using the command-line",
                    default=None,
                    nargs=argparse.REMAINDER)

args = parser.parse_args(args=['--cfg','D:/desktop/1\z/feature/transformer/original-global-plus-local-fp-transformer/experiments/global_plus_local-inference.yaml',
                               '--local_rank','0', '--port','9000'])

In [12]:
init_distributed(args)
setup_cudnn(config)

update_config(config, args)
final_output_dir = create_logger(config, args.cfg, 'test')
tb_log_dir = final_output_dir

if comm.is_main_process():
    output_config_path = os.path.join(final_output_dir, 'config.yaml')

model = build_model(config)

model_file = config.TEST.MODEL_FILE if config.TEST.MODEL_FILE \
    else os.path.join(final_output_dir, 'model_best.pth')
ext = model_file.split('.')[-1]
if ext == 'pth':
    state_dict = torch.load(model_file, map_location="cpu")
    if "checkpoint" in model_file:
        state_dict = state_dict['state_dict']
else:
    raise ValueError("Unknown model file")
model.load_state_dict(state_dict, strict=True)
model.to(torch.device('cuda'))

writer_dict = {
    'writer': SummaryWriter(logdir=tb_log_dir),
    'train_global_steps': 0,
    'valid_global_steps': 0,
}

summary_model_on_master(model, config, final_output_dir, False)

# define loss function (criterion) and optimizer
criterion = build_criterion(config, train=False)
criterion.cuda()

2025-04-12 04:47:55,749:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:55,749:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:55,749:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:55,757:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:55,757:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:55,757:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:55,760:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:55,760:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:55,760:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:55,765:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:55,765:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:55,765:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:55,770:[P:8712]:Rank[0/1] => 

=> merge config from D:/desktop/1\z/feature/transformer/original-global-plus-local-fp-transformer/experiments/global_plus_local-inference.yaml
=> creating D:\desktop\1\z\SOCOFing\embs ...
=> creating D:\desktop\1\z\SOCOFing\embs\imagenet\global_plus_local-inferenceglobal_plus_local-inferenceglobal_plus_local-inference ...
=> setup logger ...


2025-04-12 04:47:56,023:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:56,023:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:56,023:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:56,027:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:56,027:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:56,027:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:56,029:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:56,029:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:56,029:[P:8712]:Rank[0/1] => init weight of Linear from trunc norm
2025-04-12 04:47:56,033:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:56,033:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:56,033:[P:8712]:Rank[0/1] => init bias of Linear to zeros
2025-04-12 04:47:56,036:[P:8712]:Rank[0/1] => 

CVTDETR(
  31.812 M, 99.772% Params, 13.668 GMac, 100.000% MACs, 
  (cvt): ConvolutionalVisionTransformerWithTokenOut(
    19.667 M, 61.682% Params, 11.902 GMac, 87.079% MACs, 
    (stage0): VisionTransformer(
      0.061 M, 0.192% Params, 0.493 GMac, 3.610% MACs, 
      (patch_embed): ConvEmbed(
        0.009 M, 0.030% Params, 0.087 GMac, 0.639% MACs, 
        (proj): Conv2d(0.009 M, 0.030% Params, 0.087 GMac, 0.639% MACs, 3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(2, 2))
        (norm): LayerNorm(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, (64,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, p=0.0, inplace=False)
      (blocks): ModuleList(
        0.052 M, 0.163% Params, 0.406 GMac, 2.971% MACs, 
        (0): Block(
          0.052 M, 0.163% Params, 0.406 GMac, 2.971% MACs, 
          (norm1): LayerNorm(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, (64,), eps=1e-05, elementwise_affine=True)
          (attn)

MatchThenSupervise(
  (matcher): HungarianMatcher()
  (KDTokenLoss): MSELoss()
  (KDGlobalLoss): MSELoss()
  (PosoriLoss): MSELoss()
)

In [14]:
valid_loader = build_dataloader(config, False)

# data points in test/val dataset:  100
